In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import seaborn as sns
import numpy as np
import h5py

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.interpolate import griddata
from collections import OrderedDict

sns.set_style("white")

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

C:\Users\ameyv\anaconda3\envs\amey-pinn\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Multi-Layer-Perceptron(MLP) architecture for our Physics Informed Neural Network(PINN)
class MLP(torch.nn.Module):
    def __init__(self, layers):
        super(MLP, self).__init__()
        
        self.depth = len(layers) - 1
        self.activation = torch.nn.Tanh
        
        layer_list = list()
        for i in range(self.depth - 1): 
            layer_list.append(('layer_%d' % i, torch.nn.Linear(layers[i], layers[i+1])))
            layer_list.append(('activation_%d' % i, self.activation()))
        layer_list.append(('layer_%d' % (self.depth - 1), torch.nn.Linear(layers[-2], layers[-1])))
        
        layerDict = OrderedDict(layer_list)
        self.layers = torch.nn.Sequential(layerDict)
        
    def forward(self, xy):
        out = self.layers(xy)
        return out

In [16]:
# Class for the actual PINN containing training and other details
class PINN():
    def __init__(self, X_bc, Y_bc, S_bc, X, Y, layers, lb, ub, Re):

        # Boundary Conditions (Domain Boundaries)
        self.lb = torch.tensor(lb).float().to(device)
        self.ub = torch.tensor(ub).float().to(device)

        # Boundary Data (Dirichlet Boundary Conditions)
        self.X_bc = torch.tensor(X_bc, requires_grad=True).float().to(device)
        self.Y_bc = torch.tensor(Y_bc, requires_grad=True).float().to(device)
        self.S_bc = torch.tensor(S_bc).float().to(device)
        
        # Training Data (Colocation Points)
        self.X = torch.tensor(X, requires_grad=True).float().to(device)
        self.Y = torch.tensor(Y, requires_grad=True).float().to(device)
        
        # PINN Hyperparameters
        self.layers = layers
        self.nu = 1/Re
        self.mlp = MLP(layers).to(device)
        
        # Optimizer (LBFGS)
        self.optimizer = torch.optim.LBFGS(self.mlp.parameters(), lr=1, tolerance_grad=1e-5, tolerance_change=1.0 * np.finfo(float).eps, line_search_fn="strong_wolfe")
        self.iter = 0
        
    def netS(self, x, y):
        #x = torch.reshape(x, (x.shape[0], 1))
        #y = torch.reshape(x, (y.shape[0], 1))
        s = self.mlp(torch.cat([x, y], dim=1))
        return s
    
    def netF(self, x, y):
        s = self.netS(x, y)
        #u = torch.reshape(s[:,0], (s.shape[0], 1))
        #v = torch.reshape(s[:,1], (s.shape[0], 1))
        #p = torch.reshape(s[:,2], (s.shape[0], 1))
        # print("u: ", u.shape)
        # print("u: ", type(u))
        # print("x: ", x.shape)
        # print("x: ", type(x))

        u = s[:,0]
        v = s[:,1]
        p = s[:,2]
        
        du_dy = torch.autograd.grad(inputs=y, outputs=u, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
        dv_dy = torch.autograd.grad(inputs=y, outputs=v, grad_outputs=torch.ones_like(v), retain_graph=True, create_graph=True)[0]
        dp_dy = torch.autograd.grad(inputs=y, outputs=p, grad_outputs=torch.ones_like(p), retain_graph=True, create_graph=True)[0]
        
        du_dx = torch.autograd.grad(inputs=x, outputs=u, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
        dv_dx = torch.autograd.grad(inputs=x, outputs=v, grad_outputs=torch.ones_like(v), retain_graph=True, create_graph=True)[0]
        dp_dx = torch.autograd.grad(inputs=x, outputs=p, grad_outputs=torch.ones_like(p), retain_graph=True, create_graph=True)[0]
        
        du_dxx = torch.autograd.grad(inputs=x, outputs=du_dx, grad_outputs=torch.ones_like(du_dx), retain_graph=True, create_graph=True)[0]
        du_dyy = torch.autograd.grad(inputs=y, outputs=du_dy, grad_outputs=torch.ones_like(du_dy), retain_graph=True, create_graph=True)[0]
        dv_dxx = torch.autograd.grad(inputs=x, outputs=dv_dx, grad_outputs=torch.ones_like(dv_dx), retain_graph=True, create_graph=True)[0]
        dv_dyy = torch.autograd.grad(inputs=y, outputs=dv_dy, grad_outputs=torch.ones_like(dv_dy), retain_graph=True, create_graph=True)[0]
        
        # du_dx, du_dy = torch.autograd.grad(u, (x, y), create_graph=True)
        # dv_dx, dv_dy = torch.autograd.grad(v, (x, y), create_graph=True)
        # dp_dx, dp_dy = torch.autograd.grad(p, (x, y), create_graph=True)

        # du_dxx, du_dxy = torch.autograd.grad(du_dx[0], (x, y))
        # du_dyx, du_dyy = torch.autograd.grad(du_dy[1], (x, y))
        # dv_dxx, dv_dxy = torch.autograd.grad(dv_dx[0], (x, y))
        # dv_dyx, dv_dyy = torch.autograd.grad(dv_dy[1], (x, y))

        f1 = u.squeeze() * du_dx + v.squeeze() * du_dy + dp_dx - self.nu * (du_dxx + du_dyy)
        f2 = u.squeeze() * dv_dx + v.squeeze() * dv_dy + dp_dy - self.nu * (dv_dxx + dv_dyy)
        f = f1 + f2
        return f
    
    def lossFunc(self):
        self.optimizer.zero_grad()
        
        S_pred = self.netS(self.X_bc, self.Y_bc)
        F_pred = self.netF(self.X, self.Y)

        # print("F_pred:", type(F_pred))
        # print("S_pred:", type(S_pred))
        
        S_loss = torch.mean((self.S_bc - S_pred) ** 2)
        F_loss = torch.mean(F_pred ** 2)
        loss = S_loss + F_loss
        
        loss.backward()
        self.iter += 1
        if self.iter % 5 == 0:
            print('Iter %d, Loss: %.5e, Loss_u: %.5e, Loss_f: %.5e' % (self.iter, loss.item(), S_loss.item(), F_loss.item()))
        return loss
    
    def train(self):
        self.mlp.train()
        self.optimizer.step(self.lossFunc)
     
    def predict(self, X, Y): 
        x = torch.tensor(X, requires_grad=True).float().to(device)
        y = torch.tensor(Y, requires_grad=True).float().to(device)

        self.mlp.eval()
        s = self.netS(x, y)
        f = self.netF(x, y)
        return s, f

In [17]:
# Train the model
layers = [2, 20, 20, 20, 20, 20, 3]
lb = 0
ub = 1
Re = 20

f = h5py.File('data\colocation_points.h5', 'r')
X = f['X']
Y = f['Y']
X = np.asarray(X)
Y = np.asarray(Y)

f_bc = h5py.File('data\dirichlet_boundary_points.h5', 'r')
X_bc = f_bc['X']
Y_bc = f_bc['Y']
X_bc = np.asarray(X_bc)
Y_bc = np.asarray(Y_bc)
S_bc = torch.zeros(100,3)

print("X_bc:", X_bc.shape)
print("Y_bc:", Y_bc.shape)
print("S_bc:", S_bc.shape)
print("X:", X.shape)
print("Y:", Y.shape)

model = PINN(X_bc, Y_bc, S_bc, X, Y, layers, lb, ub, Re)
model.train()

X_bc: (100, 1)
Y_bc: (100, 1)
S_bc: torch.Size([100, 3])
X: (625, 1)
Y: (625, 1)
Hello
Iter 5, Loss: 5.24919e-05, Loss_u: 5.17907e-05, Loss_f: 7.01137e-07


C:\Users\ameyv\AppData\Local\Temp\ipykernel_7088\3750513466.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.S_bc = torch.tensor(S_bc).float().to(device)


Iter 10, Loss: 2.88218e-06, Loss_u: 2.12350e-06, Loss_f: 7.58687e-07
Iter 15, Loss: 1.34079e-06, Loss_u: 8.16764e-07, Loss_f: 5.24028e-07
Iter 20, Loss: 6.35622e-07, Loss_u: 2.88222e-07, Loss_f: 3.47400e-07


In [23]:
f = h5py.File('data\Analytical_Solutions\Re_20.h5', 'r')
#f.visit(printname)
P = f['P']
U = f['U']
V = f['V']
P = np.asarray(P)
U = np.asarray(U)
V = np.asarray(V)
U = torch.from_numpy(U)
V = torch.from_numpy(V)
P = torch.from_numpy(P)
print(type(U))
print("P: ", P.shape)
print("U: ", U.shape)
print("V: ", V.shape)
f.close()

s_test = torch.hstack((U,V,P))
s_pred, f_pred = model.predict(X, Y)
print("s_pred: ", s_pred.shape)
print("s_test: ", s_test.shape)

# Relative L2 Error
error_s = torch.linalg.norm(s_test-s_pred,2)/torch.linalg.norm(s_test,2)
print('Relative L2 Error y: %e' % (error_s))

<class 'torch.Tensor'>
P:  torch.Size([625, 1])
U:  torch.Size([625, 1])
V:  torch.Size([625, 1])
s_pred:  torch.Size([625, 3])
s_test:  torch.Size([625, 3])
Relative L2 Error y: 1.000221e+00
